# 1. Camera Calibration

This script intends to calibrate a camera using an object.\
For this implementation we will use the openCV method for calibration(https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html)

**Import the libraries**

In [1]:
from platform import python_version

print(python_version())

3.8.5


In [2]:
from matplotlib import pyplot as plt
import numpy as np
import scipy as sp
import cv2

#for LM method
from scipy.optimize import curve_fit
#matplotlib widget

In [3]:
if False:
    folder_dir = "C:/Users/UX550/OneDrive - Universidade de Aveiro/Documentos/5º ANO\Thesis/calibration_photos/calibration_2_meters_tivera/right_2/"
    img_cnt = 0
    # list of images
    list_images = os.listdir(folder_dir)
    print(list_images)

In [4]:
if False:
    folder_dir = "C:/Users/UX550/OneDrive - Universidade de Aveiro/Documentos/5º ANO\Thesis/calibration_photos/calibration_2_meters_tivera/left_2/"
    img_cnt = 0
    # list of images
    list_images = os.listdir(folder_dir)
    print(list_images)

### Run through image directory

In [ ]:

if True:
        
    import os 
    import string
    from os import listdir

    camera_name = 'left'
    # get the path/directory
    folder_dir = f"C:/Users/UX550/OneDrive - Universidade de Aveiro/Documentos/5º ANO\Thesis/calibration_photos/calibration_circle_2_f8_/{camera_name}_2/"
    img_cnt = 0
    # list of images
    list_images = os.listdir(folder_dir)
    new_list = sorted(list_images,key = lambda k: int(k.split('_')[2].split('.')[0]))

    for images in new_list:
        # check if the image ends with png
        renamed_image = camera_name + "_init_" + '%s' % img_cnt + '.png'
        print(images)
        print(renamed_image)
        # If equal, no renaming
        if(images == renamed_image):
            print("equal")
        else:
            while True:
                print(images)
                renamed_image = camera_name + "_init_" + '%s' % img_cnt + '.png'
                print(renamed_image)
                try:
                    os.rename(folder_dir + '/' + images,folder_dir + '/' + renamed_image)
                except FileExistsError:
                    img_cnt += 1
                    pass
                else:
                    break
                
            
        img_cnt += 1


## Zhang's Method
Comparing to the DLT method, the Zhang method only calculate the intrinsic parameters. We instead only need to find 5 parameters.

## 2. Calibration

## Find the homographies

In [5]:
def cornerfinding(n_imgs,patternsize,dist, img_name):
    '''
    @param
    [in] n_imgs : number of images used
    [in] patternsize : size of the checkerboard
    [in] dist : distance between squares
    [in] img_name : name of the jpg
    [out] imagecoo : imagecoordinates
    [out] worldcoo : worldcoordinates
    '''
    # number of images
    n_imgs = n_imgs
    # pattern size
    patternsize= patternsize
    #distance between squares(in mm)
    dist = dist

    # specified number of iterations are completed.
    criteria = (cv2.TERM_CRITERIA_EPS +
                cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    # Vector for 3D points
    threedpoints = []

    # Vector for 2D points
    twodpoints = []

    #3D points real world coordinates( static -> chessboard)
    # create a matrix of zeros for 48 points , each with 3 coordinates
    world_coo = np.zeros((1, patternsize[0]* patternsize[1],3), np.float32)
    world_coo[:, :, :2] = np.mgrid[0:patternsize[0], 0:patternsize[1]].T.reshape(-1, 2)
    # Transform coordinates in mm
    world_coo = world_coo*dist

    #prev_img_shape = None

    for i in range(n_imgs):

        im = i
        #Import the image
        name = (str(img_name)+ str(im)+".png")
        img = cv2.imread(name)
        #print(name)
        # convert it to gray scale
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        retval, corners= cv2.findChessboardCorners(gray,patternsize,None)
        #print(corners)
        if(retval == True):
            threedpoints.append(world_coo)
            #print(name)

            # Refining pixel coordinates or given 2d points.
            corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria).reshape(patternsize[0]*patternsize[1],2)

            twodpoints.append(corners2)
            # Draw and display the corners
            image = cv2.drawChessboardCorners(img,patternsize,corners2,retval)
            #plt.imshow(image)
            #plt.show()
        else:
            print(name)

    # image coordinates and world coordinates for the diferent images( the world coordinates are always the same)
    imagecoo = np.asarray(twodpoints)
    worldcoo = np.asarray(threedpoints) 
    return imagecoo, worldcoo

#C:\Users\UX550\OneDrive - Universidade de Aveiro\Documentos\5º ANO\Thesis\calibration\images
#name_left = ("../calibration_photos/calibration_photos_2meters/left_160/left_init_")
#name_right =("../calibration_photos/calibration_photos_2meters/right_160/right_init_")
#name_left = ("../calibration_photos/calibration_photos/left_edited/left_init_")
#name_right =("../calibration_photos/calibration_photos/right_edited/right_init_")
name_left = "../Thesis/calibration_photos/calibration_circle_2_f8_/left_2/left_init_"
name_right = ("../Thesis/calibration_photos/calibration_circle_2_f8_/right_2/right_init_")

#name_right = ("../calibration_photos/calibration_2_meters_tivera/right_2/right_init_")
#name_left = ("calibration_photos/left_edited/left_init_")
#name_right = ("calibration_photos/right_edited/right_init_")
n_imgs = 180
square_size = 44
pattern_size = (12,8)
#imagecoo_left,worldcoo_left = cornerfinding(n_imgs,pattern_size,square_size,name_left)
#imagecoo_right,worldcoo_right = cornerfinding(n_imgs,pattern_size,square_size,name_right)


In [33]:
if True:
        objp=np.array([[0,0,0],[0,1,0],[0,2,0],[0,3,0],[0,4,0],[0,5,0],[0.5,0.5,0],[1,1.5,0],[1.5,2.5,0],[2,3.5,0],[2.5,4.5,0],[3,5.5,0]],np.float32)
        for y in range(6):
                for x in range(2,16):
                        objp=np.append(objp,[np.array([objp[4*(y-2)+x][0],objp[4*(y-2)+x][1]+1,0])],axis=0)

        objp = objp*60.8 

In [34]:
objp

array([[  0. ,   0. ,   0. ],
       [  0. ,  60.8,   0. ],
       [  0. , 121.6,   0. ],
       [  0. , 182.4,   0. ],
       [  0. , 243.2,   0. ],
       [  0. , 304. ,   0. ],
       [ 30.4,  30.4,   0. ],
       [ 60.8,  91.2,   0. ],
       [ 91.2, 152. ,   0. ],
       [121.6, 212.8,   0. ],
       [152. , 273.6,   0. ],
       [182.4, 334.4,   0. ],
       [ 30.4,  91.2,   0. ],
       [ 91.2, 212.8,   0. ],
       [152. , 334.4,   0. ],
       [ 30.4, 152. ,   0. ],
       [152. , 395.2,   0. ],
       [152. , 456. ,   0. ],
       [  0. ,  60.8,   0. ],
       [  0. , 121.6,   0. ],
       [  0. , 182.4,   0. ],
       [  0. , 243.2,   0. ],
       [  0. , 304. ,   0. ],
       [  0. , 364.8,   0. ],
       [ 30.4,  91.2,   0. ],
       [ 60.8, 152. ,   0. ],
       [ 30.4, 152. ,   0. ],
       [ 30.4, 212.8,   0. ],
       [  0. ,  60.8,   0. ],
       [  0. , 121.6,   0. ],
       [  0. , 182.4,   0. ],
       [  0. , 243.2,   0. ],
       [  0. , 304. ,   0. ],
       [  

In [ ]:
objp = np.zeros((96, 3), np.float32)
objp[0]  = (0  ,   0  , 0)
objp[1]  = (0  , 60.8 , 0)
objp[2]  = (0  , 121.6, 0)
objp[3]  = (0  , 182.4, 0)
objp[4]  = (0  , 243.2, 0)
objp[5]  = (0  ,   304, 0)

objp[6]  = (30.4 , 30.4 , 0)
objp[7]  = (30.4 , 91.2., 0)
objp[8]  = (30.4 ,   152, 0)
objp[9]  = (30.4 , 212.8, 0)
objp[10] = (30.4 , 273.6, 0)
objp[11] = (30.4 , 334.4, 0)

objp[12] = (60.8 ,  0  , 0)
objp[13] = (60.8 ,60.8 , 0)
objp[14] = (60.8 ,121.6, 0)
objp[15] = (60.8 ,182.4, 0)
objp[16] = (60.8 ,243.2, 0)
objp[17] = (60.8 ,  304, 0)

objp[18] = (91.2 , 30.4 , 0)
objp[19] = (91.2 , 91.2., 0)
objp[20] = (91.2 ,   152, 0)
objp[21] = (91.2 , 212.8, 0)
objp[22] = (91.2 , 273.6, 0)
objp[23] = (91.2 , 334.4, 0)

objp[24] = (121.6,  0  , 0)
objp[25] = (121.6,60.8 , 0)
objp[26] = (121.6,121.6, 0)
objp[27] = (121.6,182.4, 0)
objp[28] = (121.6,243.2, 0)
objp[29] = (121.6,  304, 0)

objp[30] = (152,  30.4 , 0)
objp[31] = (152,  91.2., 0)
objp[32] = (152,    152, 0)
objp[33] = (152,  212.8, 0)
objp[34] = (152,  273.6, 0)
objp[35] = (152,  334.4, 0)

objp[36] = (182.4,  0  , 0)
objp[37] = (182.4,60.8 , 0)
objp[38] = (182.4,121.6, 0)
objp[39] = (182.4,182.4, 0)
objp[40] = (182.4,243.2, 0)
objp[41] = (182.4,  304, 0)

objp[42] = (212.8,  30.4 , 0)
objp[43] = (212.8,  91.2., 0)
objp[44] = (212.8,    152, 0)
objp[45] = (212.8,  212.8, 0)
objp[46] = (212.8,  273.6, 0)
objp[47] = (212.8,  334.4, 0)

objp[48] = (243.2,  0  , 0)
objp[49] = (243.2,60.8 , 0)
objp[50] = (243.2,121.6, 0)
objp[51] = (243.2,182.4, 0)
objp[52] = (243.2,243.2, 0)
objp[53] = (243.2,  304, 0)

objp[54] = (273.6,  30.4 , 0)
objp[55] = (273.6,  91.2., 0)
objp[56] = (273.6,    152, 0)
objp[57] = (273.6,  212.8, 0)
objp[58] = (273.6,  273.6, 0)
objp[59] = (273.6,  334.4, 0)

objp[60] = (304,  0  , 0)
objp[61] = (304,60.8 , 0)
objp[62] = (304,121.6, 0)
objp[63] = (304,182.4, 0)
objp[64] = (304,243.2, 0)
objp[65] = (304,  304, 0)

objp[66] = (334.4,  30.4 , 0)
objp[67] = (334.4,  91.2., 0)
objp[68] = (334.4,    152, 0)
objp[69] = (334.4,  212.8, 0)
objp[70] = (334.4,  273.6, 0)
objp[71] = (334.4,  334.4, 0)

objp[72] = (364.8,  0  , 0)
objp[73] = (364.8,60.8 , 0)
objp[74] = (364.8,121.6, 0)
objp[75] = (364.8,182.4, 0)
objp[76] = (364.8,243.2, 0)
objp[77] = (364.8,  304, 0)

objp[78] = (395.2,  30.4 , 0)
objp[79] = (395.2,  91.2., 0)
objp[80] = (395.2,    152, 0)
objp[81] = (395.2,  212.8, 0)
objp[82] = (395.2,  273.6, 0)
objp[83] = (395.2,  334.4, 0)

objp[84] = (425.6,  0  , 0)
objp[85] = (425.6,60.8 , 0)
objp[86] = (425.6,121.6, 0)
objp[87] = (425.6,182.4, 0)
objp[88] = (425.6,243.2, 0)
objp[89] = (425.6,  304, 0)

objp[90] = (456,  30.4 , 0)
objp[91] = (456,  91.2., 0)
objp[92] = (456,    152, 0)
objp[93] = (456,  212.8, 0)
objp[94] = (456,  273.6, 0)
objp[95] = (456,  334.4, 0)

In [53]:
objp = np.zeros((96, 3), np.float32)
objp[0]  = (0  , 0  , 0)
objp[1]  = (0  , 60 , 0)
objp[2]  = (0  , 120, 0)
objp[3]  = (0  , 180, 0)
objp[4]  = (0  , 240, 0)
objp[5]  = (0  , 300, 0)

objp[6]  = (30 , 30 , 0)
objp[7]  = (30 , 90 , 0)
objp[8]  = (30 , 150, 0)
objp[9]  = (30 , 210, 0)
objp[10] = (30 , 270, 0)
objp[11] = (30 , 330, 0)

objp[12] = (60 , 0  , 0)
objp[13] = (60 , 60 , 0)
objp[14] = (60 , 120, 0)
objp[15] = (60 , 180, 0)
objp[16] = (60 , 240, 0)
objp[17] = (60 , 300, 0)

objp[18] = (90 , 30 , 0)
objp[19] = (90 , 90 , 0)
objp[20] = (90 , 150, 0)
objp[21] = (90 , 210, 0)
objp[22] = (90 , 270, 0)
objp[23] = (90 , 330, 0)

objp[24] = (120, 0  , 0)
objp[25] = (120, 60 , 0)
objp[26] = (120, 120, 0)
objp[27] = (120, 180, 0)
objp[28] = (120, 240, 0)
objp[29] = (120, 300, 0)

objp[30] = (150, 30 , 0)
objp[31] = (150, 90 , 0)
objp[32] = (150, 150, 0)
objp[33] = (150, 210, 0)
objp[34] = (150, 270, 0)
objp[35] = (150, 330, 0)

objp[36] = (180, 0  , 0)
objp[37] = (180, 60 , 0)
objp[38] = (180, 120, 0)
objp[39] = (180, 180, 0)
objp[40] = (180, 240, 0)
objp[41] = (180, 300, 0)

objp[42] = (210, 30, 0)
objp[43] = (210, 90, 0)
objp[44] = (210, 150, 0)
objp[45] = (210, 210, 0)
objp[46] = (210, 270, 0)
objp[47] = (210, 330, 0)

objp[48] = (240, 0  , 0)
objp[49] = (240, 60 , 0)
objp[50] = (240, 120, 0)
objp[51] = (240, 180, 0)
objp[52] = (240, 240, 0)
objp[53] = (240, 300, 0)

objp[54] = (270, 30, 0)
objp[55] = (270, 90, 0)
objp[56] = (270, 150, 0)
objp[57] = (270, 210, 0)
objp[58] = (270, 270, 0)
objp[59] = (270, 330, 0)

objp[60] = (300, 0  , 0)
objp[61] = (300, 60 , 0)
objp[62] = (300, 120, 0)
objp[63] = (300, 180, 0)
objp[64] = (300, 240, 0)
objp[65] = (300, 300, 0)

objp[66] = (330, 30, 0)
objp[67] = (330, 90, 0)
objp[68] = (330, 150, 0)
objp[69] = (330, 210, 0)
objp[70] = (330, 270, 0)
objp[71] = (330, 330, 0)

objp[72] = (360, 0  , 0)
objp[73] = (360, 60 , 0)
objp[74] = (360, 120, 0)
objp[75] = (360, 180, 0)
objp[76] = (360, 240, 0)
objp[77] = (360, 300, 0)

objp[78] = (390, 30, 0)
objp[79] = (390, 90, 0)
objp[80] = (390, 150, 0)
objp[81] = (390, 210, 0)
objp[82] = (390, 270, 0)
objp[83] = (390, 330, 0)

objp[84] = (410, 0  , 0)
objp[85] = (410, 60 , 0)
objp[86] = (410, 120, 0)
objp[87] = (410, 180, 0)
objp[88] = (410, 240, 0)
objp[89] = (410, 300, 0)

objp[90] = (440, 30, 0)
objp[91] = (440, 90, 0)
objp[92] = (440, 150, 0)
objp[93] = (440, 210, 0)
objp[94] = (440, 270, 0)
objp[95] = (440, 330, 0)
# Arrays to store object points and image points from all the images.


In [54]:
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

########################################Blob Detector##############################################

# Setup SimpleBlobDetector parameters.
blobParams = cv2.SimpleBlobDetector_Params()

# Change thresholds
blobParams.minThreshold = 230
blobParams.maxThreshold = 255

# Filter by Area.
blobParams.filterByArea = True
blobParams.minArea = 60     # minArea may be adjusted to suit for your experiment
blobParams.maxArea = 250   # maxArea may be adjusted to suit for your experiment

# Filter by Circularity
blobParams.filterByCircularity = True
blobParams.minCircularity = 0.8

# Filter by Convexity
blobParams.filterByConvexity = True
blobParams.minConvexity = 0.87

# Filter by Inertia
blobParams.filterByInertia = True
blobParams.minInertiaRatio = 0.01

# Create a detector with the parameters
blobDetector = cv2.SimpleBlobDetector_create(blobParams)

## Left camera 

In [55]:
nimage = 60
imgpoints_L = np.zeros((nimage,objp.shape[0],2), np.float32) # 2d points in image plane.
objpoints = np.zeros((1,objp.shape[0],3),np.float32) 
objpoints = objp
threepoints = []

import os 
for i in range(0,nimage,1):

    name = (str(name_left)+str(i)+".png")
    #print(name)
    found = 0
    img = cv2.imread(name)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    keypoints = blobDetector.detect(gray) # Detect blobs

    im_with_keypoints = cv2.drawKeypoints(img, keypoints, np.array([]), (0,255,0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    im_with_keypoints_gray = cv2.cvtColor(im_with_keypoints, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findCirclesGrid(im_with_keypoints, (6,16), None, flags = cv2.CALIB_CB_ASYMMETRIC_GRID)   # Find the circle grid

    if ret == True:
          # Certainly, every loop objp is the same, in 3D.

        corners2 = cv2.cornerSubPix(im_with_keypoints_gray, corners, (11,11), (-1,-1), criteria).reshape(16*6,2)    # Refines the corner locations.
        imgpoints_L[i,:,:] = corners2
        threepoints.append(objpoints)

        # Draw and display the corners.
        im_with_keypoints = cv2.drawChessboardCorners(img, (6,16), corners2, ret)
        found += 1

    else:
        print(f"false: {name}\n")
        os.remove(name)
        name_right_i = (str(name_right)+str(i)+".png")
        os.remove(name_right_i)
        


        
    #cv2.imshow("img",im_with_keypoints)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

worldpoints = np.asarray(threepoints,np.float32).reshape(nimage,1,96,3)




In [56]:
worldpoints.shape

(60, 1, 96, 3)

In [57]:
type(imgpoints_L[0,0,0])

numpy.float32

## Right camera

In [58]:
nimage = 60
imgpoints_R = np.zeros((nimage,objp.shape[0],2),np.float32) # 2d points in image plane.

for i in range(0,nimage,1):

    name = (str(name_right)+str(i)+".png")
    found = 0
    img = cv2.imread(name)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    keypoints_R = blobDetector.detect(gray) # Detect blobs

    im_with_keypoints_R = cv2.drawKeypoints(img, keypoints_R, np.array([]), (0,255,0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    im_with_keypoints_gray_R = cv2.cvtColor(im_with_keypoints_R, cv2.COLOR_BGR2GRAY)
    ret_R, corners_R = cv2.findCirclesGrid(im_with_keypoints_R, (6,16), None, flags = cv2.CALIB_CB_ASYMMETRIC_GRID)   # Find the circle grid
    if ret_R == True:
        cornersR = cv2.cornerSubPix(im_with_keypoints_gray_R, corners, (11,11), (-1,-1), criteria).reshape(16*6,2)    # Refines the corner locations.
        imgpoints_R[i,:,:] = cornersR

        # Draw and display the corners.
        im_with_keypoints_R = cv2.drawChessboardCorners(img, (6,16), corners_R, ret_R)
        found += 1

    else:
        print(f"false: {name}\n")
        os.remove(name)
        name_left_i = (str(name_left)+str(i)+".png")
        os.remove(name_left_i)

    #cv2.imshow("img",im_with_keypoints_R)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()



In [59]:
print(np.shape(imgpoints_L))
print(np.shape(imgpoints_R))

(60, 96, 2)
(60, 96, 2)


In [60]:
type(worldpoints[0,0,0,0])

numpy.float32

In [61]:
def calib(world_pts_arr,x_zhang_R,x_zhang_L,image_size):    
    ret_R, mtx_R, dist_R, rvecs_R, tvecs_R = cv2.calibrateCamera(world_pts_arr, x_zhang_R, (image_size[0],image_size[1]), None, None)
    ret_L, mtx_L, dist_L, rvecs_L, tvecs_L = cv2.calibrateCamera(world_pts_arr, x_zhang_L, (image_size[0],image_size[1]), None, None)

    flags = 0
    flags |= cv2.CALIB_FIX_INTRINSIC
    # Here we fix the intrinsic camara matrixes so that only Rot, Trns, Emat and Fmat are calculated.
    # Hence intrinsic parameters are the same 

    criteria_stereo= (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)


    # This step is performed to transformation between the two cameras and calculate Essential and Fundamenatl matrix
    retS, new_mtxL, distL, new_mtxR, distR, Rot, Trns, Emat, Fmat = cv2.stereoCalibrate(world_pts_arr, x_zhang_L, x_zhang_R, mtx_L, dist_L, mtx_R, dist_R, (image_size[0],image_size[1]), criteria_stereo, flags)

    return ret_R,mtx_R,ret_L,mtx_L,retS,new_mtxL,new_mtxR,Rot,Trns,Emat,Fmat,distL,distR,rvecs_L,rvecs_R

In [68]:
ret_R,mtx_R,ret_L,mtx_L,retS,new_mtxL,new_mtxR,Rot,Trns,Emat,Fmat,distL,distR,rvecs_L,rvecs_R = calib(worldpoints,imgpoints_R,imgpoints_L,[1216,1936])

In [67]:
gray.shape[::-1]

(1936, 1216)

In [69]:
new_mtxL

array([[7.52880331e+03, 0.00000000e+00, 5.04870364e+02],
       [0.00000000e+00, 6.74215121e+03, 1.01897411e+03],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [70]:
new_mtxR

array([[4.35057095e+03, 0.00000000e+00, 6.56004986e+02],
       [0.00000000e+00, 3.11386213e+03, 6.60487703e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [66]:
Trns

array([[ 1197.57682508],
       [ 2691.02479483],
       [-1500.61293136]])

In [20]:
np.shape(imgpoints_L)

(30, 96, 2)

In [ ]:
P_L_list = []
P_R_list = []
R1_list = []
R2_list = []
R1,R2,P_L_est,P_R_est,Q,roi_left,roi_right = cv2.stereoRectify(new_mtxL, ret_L, new_mtxR, ret_R, (image_size[0],image_size[1]), Rot, Trns,flags = cv2.CALIB_ZERO_DISPARITY)

R1_list.append(R1)
R2_list.append(R2)
P_L_list.append(P_L_est)
P_R_list.append(P_R_est)

In [ ]:
new_mtxL

In [ ]:
np.savetxt(f"../calibration_photos/calibration_2_meters_tivera/calib_2_5.txt",R1,R2,P_L_est,P_R_est,retS,new_mtxL,new_mtxR,ret_L,ret_R,dist_L,dist_R,Rot,Trns)

In [ ]:
P_L_est

In [ ]:
mtx_L

In [ ]:
mtx_R